In [1]:
import os
import torch
import torch.nn.functional as F
import numpy as np
import time
from typing import Callable, Union

In [2]:
from fms.models.llama import LLaMAConfig, LLaMA

In [3]:
modelc = LLaMAConfig(128, 512, 1e-6, 16, 0, 8)
model = LLaMA(modelc)

In [4]:
from fms.utils.generation import generate

inp = torch.arange(8).unsqueeze(0)

start = time.time()
oracle = generate(model, inp, 1024, 1024, do_sample=False, use_cache=True)
print(time.time()-start)

# oracle.squeeze()

15.691529989242554


In [9]:
start = time.time()
out = speculative_generate(model, inp, 1024, 1024)
print(time.time()-start)

Speculation: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   0,   0]])
Verification: tensor([[ 58, 119,   5,  42,   5]]) 2
Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5]])

Speculation: tensor([[5, 0, 0, 0, 0]])
Verification: tensor([[10,  5,  5,  5, 54]]) 0
Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5,  10]])

Speculation: tensor([[10, 86, 79, 10, 42]])
Verification: tensor([[86, 79, 10, 42,  5]]) 4
Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5,  10,  86,  79,
          10,  42,   5]])

Speculation: tensor([[5, 0, 0, 0, 0]])
Verification: tensor([[119,  10, 107, 107,  79]]) 0
Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5,  10,  86,  79,
          10,  42,   5, 119]])

Speculation: tensor([[119,  71,  35,   0,   0]])
Verification: tensor([[71, 35, 81, 95, 95]]) 2
Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5, 

Verification: tensor([[ 75,  75,  69,  97, 119]]) 2
Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5,  10,  86,  79,
          10,  42,   5, 119,  71,  35,  81, 108, 115, 119, 126, 121,   5,  81,
         104,  35, 119, 115,  73,  68,  73,  68,  73,  45, 119,  97,  97,  97,
          37, 106,  79, 107, 111,  35,  37, 125,  68, 115,  99,   4, 106,  95,
          92, 125,  13, 120,  42,  11,  81, 114, 120,  42, 115,  51,  59, 114,
         125,  13,  73,  45, 115, 106,  10,  81,   4,  61, 103,  50, 121,  32,
           0,  75,  75,  75,  75,  75,  75,  69]])

Speculation: tensor([[ 69, 100,  81,   0,   0]])
Verification: tensor([[100,  81,  81,  75,  81]]) 2
Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5,  10,  86,  79,
          10,  42,   5, 119,  71,  35,  81, 108, 115, 119, 126, 121,   5,  81,
         104,  35, 119, 115,  73,  68,  73,  68,  73,  45, 119,  97,  97,  97,
          37, 106,  79, 107, 111,  35,  37, 125,  68

Verification: tensor([[ 59,  49, 125,  97,   4]]) 3
Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5,  10,  86,  79,
          10,  42,   5, 119,  71,  35,  81, 108, 115, 119, 126, 121,   5,  81,
         104,  35, 119, 115,  73,  68,  73,  68,  73,  45, 119,  97,  97,  97,
          37, 106,  79, 107, 111,  35,  37, 125,  68, 115,  99,   4, 106,  95,
          92, 125,  13, 120,  42,  11,  81, 114, 120,  42, 115,  51,  59, 114,
         125,  13,  73,  45, 115, 106,  10,  81,   4,  61, 103,  50, 121,  32,
           0,  75,  75,  75,  75,  75,  75,  69, 100,  81,  81,  81,  81,  97,
          97,  97,  97,  97,  97,  97,  97, 107,  64,   2,  64,   2,  64,   2,
          64,   2,  22,  34,  64,   2,  27, 119, 105,  70,  59,  49, 125,  97]])

Speculation: tensor([[97,  0,  0,  0,  0]])
Verification: tensor([[ 18,   4,  87,   1, 102]]) 0
Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5,  10,  86,  79,
          10,  42,   5, 119

Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5,  10,  86,  79,
          10,  42,   5, 119,  71,  35,  81, 108, 115, 119, 126, 121,   5,  81,
         104,  35, 119, 115,  73,  68,  73,  68,  73,  45, 119,  97,  97,  97,
          37, 106,  79, 107, 111,  35,  37, 125,  68, 115,  99,   4, 106,  95,
          92, 125,  13, 120,  42,  11,  81, 114, 120,  42, 115,  51,  59, 114,
         125,  13,  73,  45, 115, 106,  10,  81,   4,  61, 103,  50, 121,  32,
           0,  75,  75,  75,  75,  75,  75,  69, 100,  81,  81,  81,  81,  97,
          97,  97,  97,  97,  97,  97,  97, 107,  64,   2,  64,   2,  64,   2,
          64,   2,  22,  34,  64,   2,  27, 119, 105,  70,  59,  49, 125,  97,
          18, 103, 107,  63,  18,  55, 103,  73,  48,  75,  75,  33,  97, 103,
         107,  64,  32,  42,  22, 120,  29,  75,  75,  75,  75,  75,  75,  75,
          75,  75,  75,  75,  75,  75,  33,  97,  97,  97, 103]])

Speculation: tensor([[103, 107,   4,   0,   0]])

Verification: tensor([[97, 11, 97, 11, 81]]) 3
Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5,  10,  86,  79,
          10,  42,   5, 119,  71,  35,  81, 108, 115, 119, 126, 121,   5,  81,
         104,  35, 119, 115,  73,  68,  73,  68,  73,  45, 119,  97,  97,  97,
          37, 106,  79, 107, 111,  35,  37, 125,  68, 115,  99,   4, 106,  95,
          92, 125,  13, 120,  42,  11,  81, 114, 120,  42, 115,  51,  59, 114,
         125,  13,  73,  45, 115, 106,  10,  81,   4,  61, 103,  50, 121,  32,
           0,  75,  75,  75,  75,  75,  75,  69, 100,  81,  81,  81,  81,  97,
          97,  97,  97,  97,  97,  97,  97, 107,  64,   2,  64,   2,  64,   2,
          64,   2,  22,  34,  64,   2,  27, 119, 105,  70,  59,  49, 125,  97,
          18, 103, 107,  63,  18,  55, 103,  73,  48,  75,  75,  33,  97, 103,
         107,  64,  32,  42,  22, 120,  29,  75,  75,  75,  75,  75,  75,  75,
          75,  75,  75,  75,  75,  75,  33,  97,  97,  97, 103, 107,

...

Verification: tensor([[ 58,  58, 112, 112, 112]]) 1
Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5,  10,  86,  79,
          10,  42,   5, 119,  71,  35,  81, 108, 115, 119, 126, 121,   5,  81,
         104,  35, 119, 115,  73,  68,  73,  68,  73,  45, 119,  97,  97,  97,
          37, 106,  79, 107, 111,  35,  37, 125,  68, 115,  99,   4, 106,  95,
          92, 125,  13, 120,  42,  11,  81, 114, 120,  42, 115,  51,  59, 114,
         125,  13,  73,  45, 115, 106,  10,  81,   4,  61, 103,  50, 121,  32,
           0,  75,  75,  75,  75,  75,  75,  69, 100,  81,  81,  81,  81,  97,
          97,  97,  97,  97,  97,  97,  97, 107,  64,   2,  64,   2,  64,   2,
          64,   2,  22,  34,  64,   2,  27, 119, 105,  70,  59,  49, 125,  97,
          18, 103, 107,  63,  18,  55, 103,  73,  48,  75,  75,  33,  97, 103,
         107,  64,  32,  42,  22, 120,  29,  75,  75,  75,  75,  75,  75,  75,
          75,  75,  75,  75,  75,  75,  33,  97,  97,  97, 103,

Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5,  10,  86,  79,
          10,  42,   5, 119,  71,  35,  81, 108, 115, 119, 126, 121,   5,  81,
         104,  35, 119, 115,  73,  68,  73,  68,  73,  45, 119,  97,  97,  97,
          37, 106,  79, 107, 111,  35,  37, 125,  68, 115,  99,   4, 106,  95,
          92, 125,  13, 120,  42,  11,  81, 114, 120,  42, 115,  51,  59, 114,
         125,  13,  73,  45, 115, 106,  10,  81,   4,  61, 103,  50, 121,  32,
           0,  75,  75,  75,  75,  75,  75,  69, 100,  81,  81,  81,  81,  97,
          97,  97,  97,  97,  97,  97,  97, 107,  64,   2,  64,   2,  64,   2,
          64,   2,  22,  34,  64,   2,  27, 119, 105,  70,  59,  49, 125,  97,
          18, 103, 107,  63,  18,  55, 103,  73,  48,  75,  75,  33,  97, 103,
         107,  64,  32,  42,  22, 120,  29,  75,  75,  75,  75,  75,  75,  75,
          75,  75,  75,  75,  75,  75,  33,  97,  97,  97, 103, 107,   4,  11,
          64,  75,  75,  75,  75,  7

Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5,  10,  86,  79,
          10,  42,   5, 119,  71,  35,  81, 108, 115, 119, 126, 121,   5,  81,
         104,  35, 119, 115,  73,  68,  73,  68,  73,  45, 119,  97,  97,  97,
          37, 106,  79, 107, 111,  35,  37, 125,  68, 115,  99,   4, 106,  95,
          92, 125,  13, 120,  42,  11,  81, 114, 120,  42, 115,  51,  59, 114,
         125,  13,  73,  45, 115, 106,  10,  81,   4,  61, 103,  50, 121,  32,
           0,  75,  75,  75,  75,  75,  75,  69, 100,  81,  81,  81,  81,  97,
          97,  97,  97,  97,  97,  97,  97, 107,  64,   2,  64,   2,  64,   2,
          64,   2,  22,  34,  64,   2,  27, 119, 105,  70,  59,  49, 125,  97,
          18, 103, 107,  63,  18,  55, 103,  73,  48,  75,  75,  33,  97, 103,
         107,  64,  32,  42,  22, 120,  29,  75,  75,  75,  75,  75,  75,  75,
          75,  75,  75,  75,  75,  75,  33,  97,  97,  97, 103, 107,   4,  11,
          64,  75,  75,  75,  75,  7

Verification: tensor([[72, 44, 32, 32, 32]]) 1
Updated output: tensor([[  0,   1,   2,   3,   4,   5,   6,   7,  58, 119,   5,  10,  86,  79,
          10,  42,   5, 119,  71,  35,  81, 108, 115, 119, 126, 121,   5,  81,
         104,  35, 119, 115,  73,  68,  73,  68,  73,  45, 119,  97,  97,  97,
          37, 106,  79, 107, 111,  35,  37, 125,  68, 115,  99,   4, 106,  95,
          92, 125,  13, 120,  42,  11,  81, 114, 120,  42, 115,  51,  59, 114,
         125,  13,  73,  45, 115, 106,  10,  81,   4,  61, 103,  50, 121,  32,
           0,  75,  75,  75,  75,  75,  75,  69, 100,  81,  81,  81,  81,  97,
          97,  97,  97,  97,  97,  97,  97, 107,  64,   2,  64,   2,  64,   2,
          64,   2,  22,  34,  64,   2,  27, 119, 105,  70,  59,  49, 125,  97,
          18, 103, 107,  63,  18,  55, 103,  73,  48,  75,  75,  33,  97, 103,
         107,  64,  32,  42,  22, 120,  29,  75,  75,  75,  75,  75,  75,  75,
          75,  75,  75,  75,  75,  75,  33,  97,  97,  97, 103, 107,

Updated output: tensor([[ 0,  1,  2,  ..., 72,  3, 50]])

Speculation: tensor([[50, 44, 72, 44,  0]])
Verification: tensor([[44, 72, 44, 72, 32]]) 3
Updated output: tensor([[ 0,  1,  2,  ..., 72, 44, 72]])

Speculation: tensor([[72, 72,  0,  0,  0]])
Verification: tensor([[72, 72, 79, 32, 44]]) 1
Updated output: tensor([[ 0,  1,  2,  ..., 72, 72, 72]])

Speculation: tensor([[72, 72, 72, 72,  0]])
Verification: tensor([[72, 72, 72,  3, 44]]) 3
Updated output: tensor([[ 0,  1,  2,  ..., 72, 72,  3]])

Speculation: tensor([[3, 0, 0, 0, 0]])
Verification: tensor([[50, 44, 44, 37, 37]]) 0
Updated output: tensor([[ 0,  1,  2,  ..., 72,  3, 50]])

Speculation: tensor([[50, 44, 72,  0,  0]])
Verification: tensor([[44, 72, 72, 37, 37]]) 2
Updated output: tensor([[ 0,  1,  2,  ..., 44, 72, 72]])

Speculation: tensor([[72, 44, 85, 37,  0]])
Verification: tensor([[ 44,  85,  37, 125,  32]]) 3
Updated output: tensor([[  0,   1,   2,  ...,  85,  37, 125]])

Speculation: tensor([[125,  29,  63,  30, 

In [11]:
for i in range(len(out)):
    assert out[0,i]==oracle[0,i], i
print("Exact match!")

Exact match!


In [5]:
def blackbox_oracle(inp):
    out = oracle[:,inp.size(1):inp.size(1)+4].clone()
    nzero = torch.randint(5,(1,))
    if nzero > 0:
        out[:,-nzero:] = 0
    return out, min(oracle.size(1), inp.size(1)+4) - inp.size(1)

def speculative_generate(
    model: Union[Callable, torch.nn.Module],
    input_ids: torch.LongTensor,
    max_seq_len: int = 2048,
    max_new_tokens: int = 256,
    temperature: float = 1.0,
    top_k: int = 10,
    num_beams: int = 1,
    smallmodel: Callable = blackbox_oracle,
):
    do_sample = False
    use_cache = True
    batched = False
    if num_beams != 1:
        raise NotImplementedError("generate() does yet not support beam search")
    if type(input_ids) == torch.Tensor:
        if input_ids.dim() != 1:
            batched = True
    else:
        raise RuntimeError("generate() requires a tensor of token ids as the prefix")

    if not batched:
        input_ids = input_ids.unsqueeze(0)

    result = input_ids
    next_input = input_ids
    kwargs = dict()
    kwargs["past_key_value_states"] = None
    kwargs["use_cache"] = use_cache

    n_gen = 0
    while n_gen < max_new_tokens:
        input_ids = next_input[:, -max_seq_len:]
        adds, n_adds = smallmodel(result)
        input_ids = torch.cat([input_ids, adds], dim=-1)
        print("Speculation:", input_ids)
        output = model.forward(input_ids, **kwargs)
        if use_cache:
            logits, past_key_value_states = output
        else:
            logits = output
        logits = logits[:, -n_adds-1:, :]

        if do_sample:
            # get logits from last value in sequence and scale
#             logits = logits / temperature
#             if top_k:
#                 v, _ = torch.topk(logits, top_k)
#                 logits[logits < v[:, [-1]]] = -float("inf")

#             probs = F.softmax(logits, dim=-1)
#             next_val = torch.multinomial(probs, num_samples=1)
            pass
        else:
            next_vals = torch.argmax(logits, dim=-1)
        
        # Check correctness of smallmodel predictions
        n_correct = 0
        while n_correct < n_adds and next_vals[0,n_correct] == input_ids[0,-n_adds+n_correct]:
            n_correct += 1
        print("Verification:", next_vals, n_correct)
        
        # Toss any wrong smallmodel outputs
        next_vals = next_vals[:,:n_correct+1]
        n_gen += n_correct+1
            
        n_wrong = n_adds - n_correct
        if use_cache:
            # kv updates are required for torch.compile with
            # mode='reduce-overhead'
            n_kv_s = []
            for layer_idx in range(len(past_key_value_states)):
                n_kv_s.append([])
                for tensor_idx in range(len(past_key_value_states[layer_idx])):
                    base = past_key_value_states[layer_idx][tensor_idx]
                    if n_wrong > 0:
                        base = base[:,:,:-n_wrong]
                    n_kv_s[layer_idx].append(
                        base.clone(memory_format=torch.contiguous_format).detach()
                    )
                    # torch._dynamo.mark_dynamic(n_kv_s[layer_idx][tensor_idx], 2)
            kwargs["past_key_value_states"] = n_kv_s

        result = torch.cat((result, next_vals), dim=-1)
        print("Updated output:", result)
        print()

        if use_cache:
            next_input = next_vals[:,-1].unsqueeze(-1)
        else:
            next_input = result

    if not batched:
        result = result[0]
    return result